# Artificial Neural Networks for Complete Intersection Calabi-Yau Manifold

In the framework of String Theory, we apply **deep learning** techniques for the prediction of the **Hodge numbers** of _Complete Intersection Calabi-Yau_ (CICY) 3-folds. The relevant quantities are therefore $h_{11}$ and $h_{21}$ which can be predicted starting from the configuration matrices of known manifolds.

We take advantage of previously studied and feature engineered datasets to build **deep neural networks** (DNN) and **convolutional neural networks** (CNN) to predict the labels. We use [Tensorflow](https://www.tensorflow.org/) as backend framework and its high level API [Keras](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras).

## Setup

We first setup the environment and import relevant packages which we will use in the analysis. We print their versions to keep track of changes and set the **random seed** of all random generators in order to get reproducible results.

In [1]:
import sys

import matplotlib        as mpl
import matplotlib.pyplot as plt
import random            as rnd
import numpy             as np
import pandas            as pd
import sklearn           as skl
import skopt             as sko
import tensorflow        as tf
import xgboost           as xgb

from tensorflow       import keras
from tensorflow.keras import backend as K

import warnings
warnings.simplefilter(action='ignore', category=UserWarning) # ignore UserWarning: I cannot really do anything about it...

print('Python version: {:d}.{:d}'      .format(sys.version_info.major, sys.version_info.minor), flush=True)
print('Matplot version: {}'            .format(mpl.__version__),                                flush=True)
print('Numpy version: {}'              .format(np.__version__),                                 flush=True)
print('Pandas version: {}'             .format(pd.__version__),                                 flush=True)
print('Scikit-learn version: {}'       .format(skl.__version__),                                flush=True)
print('Scikit-optimize version: {}'    .format(sko.__version__),                                flush=True)
print('Tensorflow version: {}'         .format(tf.__version__),                                 flush=True)
print('Keras version: {} (backend: {})'.format(keras.__version__, K.backend()),                 flush=True)
print('XGBoost version: {}'            .format(xgb.__version__),                                flush=True)

# fix random_seed
RAND = 42
rnd.seed(RAND)
np.random.seed(RAND)
tf.random.set_seed(RAND)

Python version: 3.7
Matplot version: 3.2.1
Numpy version: 1.18.1
Pandas version: 1.0.3
Scikit-learn version: 0.22.2.post1
Scikit-optimize version: 0.7.4
Tensorflow version: 2.0.0
Keras version: 2.2.4-tf (backend: tensorflow)
XGBoost version: 0.90


We also print the **hardware specifications** to have a representation of the current build.

In [2]:
!echo "OS:  $(uname -o) - $(lsb_release -d| sed 's/^.*:\s*//g')"
!echo "CPU: $(lscpu| grep 'Model name'| sed 's/^.*:\s*//g')"
!echo "RAM: $(free --giga| awk '/^Mem/ {print $7}') GiB"
!echo "GPU: $(lspci | grep '3D controller' | sed 's/^.*controller:\s*//g')"

OS:  GNU/Linux - Arch Linux
CPU: Intel(R) Core(TM) i7-7700HQ CPU @ 2.80GHz
RAM: 8 GiB
GPU: NVIDIA Corporation GM108M [GeForce 940MX] (rev ff)


In order to store information and results, we create a **logger** for the current Python session and a function to print information to the log file (or the standard output, if the logger is not defined).

In [3]:
import logging

from os import path, rename
from time import strftime, gmtime

def create_logfile(filename, name='logger', level=logging.INFO):
    """
    Create a logfile and rotate old logs.
    
    Required arguments:
        filename: the name of the file or path to the log
        
    Optional arguments
        name:     the name of the log session
        level:    the level of the information stores
        
    Returns:
        the log
    """
    
    # get current time to rename strings
    ctime = strftime('.%Y%m%d.%H%M%S', gmtime())
    
    # rotate log if it already exists
    if path.isfile(filename):
        print('Rotating existing logs...', flush=True)
        rename(filename, filename + ctime)
    
    # get a logging session by name
    log = logging.getLogger(name + ctime)
    log.setLevel(level)
    
    # define format
    fmt = logging.Formatter('%(asctime)s --> %(levelname)s: %(message)s')
    
    # add the log file
    han = logging.FileHandler(filename=filename)
    han.setLevel(level)
    han.setFormatter(fmt)
    
    # add handler for standard output
    std = logging.StreamHandler(sys.stdout)
    std.setLevel(level)
    std.setFormatter(fmt)
    
    # create the output
    log.addHandler(han)
    log.addHandler(std)
    
    print('Created new log file!', flush=True)
    return log

def logprint(string, stream='info', logger=None):
    """
    Decides whether to print on the logger or the standard output.
    
    Required arguments:
        string: the string to print
    
    Optional arguments:
        stream: standard input (info) or standard error (error)
        logger: the logger (None for standard output/error)
    """
    
    if logger is not None:
        if stream == 'info':
            logger.info(string)
        elif stream == 'error':
            logger.error(string)
        else:
            logger.debug(string)
    else:
        if stream == 'info':
            sys.stdout.write(string)
        elif stream == 'error':
            sys.stderr.write(string)
        else:
            sys.stdout.write(string)

## Preparation and Tools

We first fetch the desired dataset and prepare the tools for the analysis. Specifically we need to:

1. define the names of the main **directories** and create them if non existent,
2. import the **database** and read the archive,
3. create tools for **visualisation** and **manipulation** of the dataset.

In [4]:
import logging
from os import makedirs

ROOT_DIR = '.'      # root directory
IMG_DIR  = 'img'    # image directory
MOD_DIR  = 'models' # directory of saved models
LOG_DIR  = 'log'    # directory for logs

# name of the dataset to be considered
DB_NAME = 'cicy3o'
DB_FILE = DB_NAME + '.h5'                                     # full name with extension
DB_PATH = path.join(ROOT_DIR, DB_FILE)                        # full path
DB_DIR  = 'original' if DB_NAME == 'cicy3o' else 'favourable' # subdir where to store images, models, logs

# define full paths
IMG_PATH = path.join(ROOT_DIR, IMG_DIR, DB_DIR)
MOD_PATH = path.join(ROOT_DIR, MOD_DIR, DB_DIR)
LOG_PATH = path.join(ROOT_DIR, LOG_DIR, DB_DIR)

# create directories if non existent
if not path.isdir(IMG_PATH):
    makedirs(IMG_PATH, exist_ok=True)
if not path.isdir(MOD_PATH):
    makedirs(MOD_PATH, exist_ok=True)
if not path.isdir(LOG_PATH):
    makedirs(LOG_PATH, exist_ok=True)
    
# create logfile
logger = create_logfile(filename=path.join(LOG_PATH, DB_NAME + '_nn.log'), name='CICY3', level=logging.INFO)

Created new log file!


We then prepare to load the dataset and prepare for the visualisation analysis.

In [5]:
import pandas as pd

def load_dataset(filepath, mode='hdf5', shuffle=False, random_state=None, logger=None):
    """
    Load a dataset given the path and the format.
    
    Required arguments:
        filepath: the path of the file
        
    Optional arguments:
        mode:         the format of the file
        shuffle:      whether to shuffle the file
        random_state: the seed of the random generator
        logger:       the logging session (None for standard output)
        
    Returns:
        the dataset
    """
    
    if path.isfile(filepath):
        logprint('Reading database...', logger=logger)
        if mode == 'hdf5':
            df = pd.read_hdf(filepath)
        elif mode == 'csv':
            df = pd.read_csv(filepath)
        logprint('Database loaded!', logger=logger)
    else:
        logprint('Database is not available: cannot load the database!', stream='error', logger=logger)
        
    # shuffle the dataframe
    if shuffle and random_state is not None:
        logprint('Shuffling database...', logger=logger)
        df = skl.utils.shuffle(df, random_state=random_state)
        logprint('Database shuffled!', logger=logger)
        
    return df

We then define some functions we can use to extract and manipulate the database. We use the _Scikit-learn_ API to create _Estimator_ classes (inheriting the _Scikit_ interface).

In [6]:
from sklearn.base import BaseEstimator, TransformerMixin

# remove the outliers from a Pandas dataset
class RemoveOutliers(BaseEstimator, TransformerMixin):
    """
    Remove outlying data given a dataset and a dictionary containing the intervals for each class.
    
    E.g.: if the two classes are 'h11' and 'h21', the dictionary will be: {'h11': [1, 16], 'h21': [1, 86]}.
    
    Public methods:
        fit:           unused method
        transform:     remove data outside the given interval
        fit_transform: equivalent to transform(fit(...))
    """

    def __init__(self, filter_dict=None):
        """
        Constructor of the class.
        
        Optional arguments:
            filter_dict: the intervals to retain in the data
        """
        
        self.filter_dict = filter_dict

    def fit(self, X, y=None):
        """
        Unused method.
        """

        return self

    def transform(self, X):
        """
        Transform the input by deleting data outside the interval
        
        Required arguments:
            X: the dataset
            
        Returns:
            the transformed dataset
        """

        x = X.copy() # avoid overwriting

        if self.filter_dict is not None:
            for key in self.filter_dict:
                x = x.loc[x[key] >= self.filter_dict[key][0]]
                x = x.loc[x[key] <= self.filter_dict[key][1]]

        return x

# extract the tensors from a Pandas dataset
class ExtractTensor(BaseEstimator, TransformerMixin):
    """
    Extract a dense tensor from sparse input from a given dataset.
    
    Public methods:
        fit:           unused method
        transform:     extract dense tensor
        fit_transform: equivalent to transform(fit(...))
        get_shape:     compute the shape of the tensor
    """

    def __init__(self, flatten=False, shape=None):
        """
        Constructor of the class.
        
        Optional arguments:
            flatten: whether to flatten the output or keep the current shape
            shape:   force the computation with a given shape
        """

        self.flatten = flatten
        self.shape   = shape

    def fit(self, X, y=None):
        """
        Unused method.
        """

        return self

    def transform(self, X):
        """
        Compute the dense equivalent of the sparse input.
        
        Required arguments:
            X: the dataset
            
        Returns:
            the transformed input
        """

        x = X.copy() # avoid overwriting
        if self.shape is None:
            self.shape = x.apply(np.shape).max() # get the shape of the tensor

        if len(self.shape) > 0: # apply this to vectors and tensors
            offset = lambda s : [ (0, self.shape[i] - np.shape(s)[i]) for i in range(len(self.shape)) ]
            x      = x.apply(lambda s: np.pad(s, offset(s), mode='constant'))

        if self.flatten and len(self.shape) > 0:
            return list(np.stack(x.apply(np.ndarray.flatten).values))
        else:
            return list(np.stack(x.values))

    def get_shape(self):
        """
        Compute the shape of the tensor.
        
        Returns:
            the shape of the tensor
        """
        
        return self.shape

We the define the functions we will use to evaluate and improve the algorithms. Even though the ultimate goal is a regression task, we will however predict integer values $h_{11},~h_{21} \in \mathbb{Z}$, thus we will evaluate the **accuracy** of the prediction, given the best estimate (in general we use the _mean squared error_ to evaluate the algorithms, but we accept those with best _accuracy_).

In [7]:
# get the accuracy (possibly after rounding)
def accuracy_score(y_true, y_pred, rounding=np.rint):
    """
    Compute the accuracy of the predictions after rounding.
    
    Required arguments:
        y_true: true values
        y_pred: predicted values
        
    Optional arguments:
        rounding: the Numpy function for rounding the predictions
    """

    assert np.shape(y_true)[0] == np.shape(y_pred)[0] # check if same length
    
    # if same length then proceed
    accuracy = 0
    if rounding is not None:
        for n in range(np.shape(y_true)[0]):
            accuracy = accuracy + 1 \
                       if int(y_true[n]) == int(rounding(y_pred[n])) \
                       else accuracy
    else:
        for n in range(np.shape(y_true)[0]):
            accuracy = accuracy + 1 \
                       if y_true[n] == y_pred[n] \
                       else accuracy
    return accuracy / np.shape(y_true)[0]

# get the error difference (possibly after rounding)
def error_diff(y_true, y_pred, rounding=np.rint):
    """
    Compute the error difference between true values and predictions (positive values are overestimate and viceversa).
    
    Required arguments:
        y_true: true values
        y_pred: predicted values
        
    Optional arguments:
        rounding: the Numpy function for rounding the predictions
    """

    assert np.shape(y_true)[0] == np.shape(y_pred)[0] # check if same length

    # if same length then proceed
    err = y_true - rounding(y_pred)
    return np.array(err).astype(np.int8)

# print *SearchCV scores
def gridcv_score(estimator, rounding=np.rint, logger=None):
    """
    Print scores given by cross-validation and optimisation techniques.
    
    Required arguments:
        estimator: the estimator to be evaluated
        
    Optional arguments:
        rounding: the Numpy function for rounding the predictions
        logger:   the logging session (None for standard output)
    """
    
    best_params = estimator.best_params_              # get best parameters
    df          = pd.DataFrame(estimator.cv_results_) # dataframe with CV res.
    
    cv_best_res = df.loc[df['params'] == best_params] # get best results
    accuracy    = cv_best_res.loc[:, 'mean_test_score'].values[0]
    std         = cv_best_res.loc[:, 'std_test_score'].values[0]
    
    logprint('Best parameters: {}'.format(best_params), logger=logger)
    logprint('Accuracy ({}) of cross-validation: ({:.3f} ± {:.3f})%'.format(rounding.__name__, accuracy*100, std*100), logger=logger)
    
# print the accuracy of the predictions
def prediction_score(estimator, X, y, use_best_estimator=False, rounding=np.rint, logger=None):
    """
    Print the accuracy of the predictions.
    
    Required arguments:
        estimator: the estimator to be used for the predictions
        X: the features
        y: the labels (actual values)
        
    Optional arguments:
        use_best_estimator: whether to use the estimator.best_estimator_ or just estimator
        rounding: the Numpy function for rounding the predictions
        logger:   the logging session (None for standard output)
    """
    
    if use_best_estimator:
        estimator = estimator.best_estimator_
    
    accuracy = accuracy_score(y, estimator.predict(X), rounding=rounding)
    logprint('Accuracy ({}) of the predictions: {:.3f}%'.format(rounding.__name__, accuracy*100), logger=logger)

We use _Matplotlib_ to plot the data and define a few functions which we can use during the analysis:

In [8]:
# set label sizes
%matplotlib inline
mpl.rc('axes', labelsize=12)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# set building block sizes for the plot
mpl_width  = 6
mpl_height = 5

# save the current figure
def save_fig(filename, tight_layout=True, extension='png', resolution=96, logger=None):
    """
    Save current figure to file.
    
    Required arguments:
        filename: the name of the file where to save the figure (without extension)
        
    Optional arguments:
        tight_layout: whether to use the tight_layout
        extension:    extension of the file to use
        resolution:   resolution of the file
        logger:       the logging session (None for standard output)
    """

    filename = path.join(IMG_PATH, filename + '.' + extension)
    if tight_layout:
        plt.tight_layout()

    logprint('Saving {}...'.format(filename), logger=logger)
    plt.savefig(filename, format=extension, dpi=resolution)
    logprint('Saved {}!'.format(filename), logger=logger)

# get a generator to count the occurrencies
def get_counts(df, label, feature):
    """
    Generator to produce the count of unique occurrencies of the data.
    
    Required arguments:
        df:      the Pandas dataframe
        label:   the label to consider
        feature: the feature to consider
        
    Yields:
        [ unique feature, unique value, counts ]
    """

    for n in np.sort(df[feature].unique()):
        uniques, counts = np.unique(df[label].loc[df[feature] == n].values, return_counts=True)
        for u, c in np.c_[uniques, counts]:
            yield [ n, u, c ]

# plot histogram of occurrencies
def count_plot(ax, data, title=None, xlabel=None, ylabel='N',
               legend=None, xlog=False, ylog=False, binstep=5,
               **kwargs):
    """
    Plot histogram of occurrencies (e.g.: frequency plot).
    
    Required arguments:
        ax:   the subplot ax where to plot data
        data: the data to plot
        
    Optional arguments:
        title:    the title of the plot
        xlabel:   the label of the x axis
        ylabel:   the label of the y axis
        legend:   the label for the legend in the plot
        xlog:     whether to use the log scale on the x axis
        ylog:     whether to use the log scale on the y axis
        binstep:  the distance between adjacent bins
        **kwargs: additional arguments to pass to plt.hist
    """

    min_tick = np.min(data) if np.min(data) > -100 else -100 # set a MIN cut
    max_tick = np.max(data) if np.max(data) < 100  else 100  # set a MAX cut

    ax.grid(alpha=0.2)                   # create a grid
    ax.set_title(title)                  # set title
    ax.set_xlabel(xlabel)                # set a label for the x axis
    ax.set_ylabel(ylabel)                # set a label for the y axis
    ax.set_xticks(np.arange(min_tick,    # set no. of ticks in the x axis
                            max_tick,
                            step=binstep
                           )
                 )

    if xlog:                             # use log scale in x axis if needed
        ax.set_xscale('log')
    if ylog:                             # use log scale in y axis if needed
        ax.set_yscale('log')

    ax.hist(data,                        # create histogram using 'step' funct.
            histtype='step',
            label=legend,
            **kwargs)

    if legend is not None:               # add legend
        ax.legend(loc='best')

    return ax

# plot labeled features and their values
def label_plot(ax, data, title=None, xlabel=None, ylabel='values',
               legend=None, xlog=False, ylog=False, binstep=1,
               **kwargs):
    """
    Plot values of labelled data (e.g.: variable ranking).
    
    Required arguments:
        ax:   the subplot ax where to plot data
        data: the data to plot
        
    Optional arguments:
        title:    the title of the plot
        xlabel:   the label of the x axis
        ylabel:   the label of the y axis
        legend:   the label for the legend in the plot
        xlog:     whether to use the log scale on the x axis
        ylog:     whether to use the log scale on the y axis
        binstep:  the distance between adjacent bins
        **kwargs: additional arguments to pass to plt.plot
    """

    labels      = [f[0] for f in data]   # labels vector
    importances = [f[1] for f in data]   # importances vector
    length      = len(labels)            # length of the labels vector
    
    ax.grid(alpha=0.2)                   # create a grid
    ax.set_title(title)                  # set title
    ax.set_xlabel(xlabel)                # set a label for the x axis
    ax.set_ylabel(ylabel)                # set a label for the x axis

    ax.set_xticks(np.arange(length,      # set no. of ticks in the x axis
                            step=binstep
                           )
                 )
    ax.set_xticklabels(labels,           # set name of labels of the x axis
                       ha='right',       # horizontal alignment
                       rotation=45       # rotation of the labels
                      )

    if xlog:                             # use log scale in x axis if needed
        ax.set_xscale('log')
    if ylog:                             # use log scale in y axis if needed
        ax.set_yscale('log')

    ax.plot(np.arange(length),           # plot data
            importances,
            label=legend,
            **kwargs)

    if legend is not None:               # add legend
        ax.legend(loc='best')

    return ax

# plot the correlation matrix of a Pandas dataframe
def mat_plot(ax, matrix, labels, label='correlation matrix', **kwargs):
    """
    Plot the correlation matrix of a given dataframe.
    
    Required arguments:
        ax:     the subplot ax where to plot data
        matrix: the matrix to plot
        labels: the labels to show with the matrix
        
    Optional arguments:
        label:    the label to use for the colour bar
        **kwargs: additional arguments to pass to plt.matshow
    """
    
    ax.set_xticks(np.arange(len(labels), # set ticks for x axis
                  step=1)
                 )
    ax.set_xticklabels([''] + labels,    # set the name of the ticks
                       rotation=90
                      )

    ax.set_yticks(np.arange(len(labels), # set ticks for y axis
                  step=1)
                 )
    ax.set_yticklabels([''] + labels)    # set the name of the ticks
                    
    matshow = ax.matshow(matrix,         # show the matrix
                         vmin=-1.0,
                         vmax=1.0,
                         **kwargs
                        )
                                
    cbar = ax.figure.colorbar(matshow,   # create the colour bar
                              ax=ax,
                              fraction=0.05,
                              pad=0.05
                             )
    cbar.ax.set_ylabel(label,            # show the colour bar
                       va='bottom',      # vertical alignment
                       rotation=-90)     # rotation of the label

    return ax

# plot a scatter plot with colours and sizes
def scatter_plot(ax, data, title=None, xlabel=None, ylabel=None,
                 legend=None, xlog=False, ylog=False,
                 colour=True, size=True, colour_label='N', size_leg=0,
                 **kwargs):
    """
    Scatter plot of occurrencies with colour and size codes.
    
    Required arguments:
        ax:   the subplot ax where to plot data
        data: the data to plot
        
    Optional arguments:
        title:        the title of the plot
        xlabel:       the label of the x axis
        ylabel:       the label of the y axis
        legend:       the label for the legend in the plot
        xlog:         whether to use the log scale on the x axis
        ylog:         whether to use the log scale on the y axis
        colour:       whether to use colour codes
        size:         whether to use entries of different size
        colour_label: label to use for the colour code
        size_leg:     length of the legend of the size code
        **kwargs:     additional arguments to pass to plt.scatter
    """

    ax.grid(alpha=0.2)                   # create  a grid
    ax.set_xlabel(xlabel)                # set labels for the x axis
    ax.set_ylabel(ylabel)                # set labels for the y axis
    ax.set_title(title)                  # set title

    if xlog:                             # use log scale in x axis if needed
        ax.set_xscale('log')
    if ylog:                             # use log scale in y axis if needed
        ax.set_yscale('log')

    if colour:                           # create the plot with size and colours
        if size:
            scat = ax.scatter(data[0], data[1], s=data[2], c=data[2], **kwargs)
        else:
            scat = ax.scatter(data[0], data[1], c=data[2], **kwargs)
        cbar = ax.figure.colorbar(scat, ax=ax)
        cbar.ax.set_ylabel(colour_label, rotation=-90, va='bottom')
    else:
        if size:
            scat = ax.scatter(data[0], data[1], s=data[2], **kwargs)
        else:
            scat = ax.scatter(data[0], data[1], **kwargs)

    scat.set_label(legend)               # set label of the plot
    if size_leg:                         # add the size legend if needed
        handles, labels = scat.legend_elements('sizes', num=size_leg)
        ax.legend(handles, labels, loc='lower center',
                  bbox_to_anchor=(0.5,-0.3), ncol=len(handles),
                  fontsize='medium', frameon=False)

    if legend:                           # show the legend
        ax.legend(loc='best')

    return ax

# plot a series with trivial x label
def series_plot(ax, data, title=None, xlabel='series', ylabel=None,
                legend=None, xlog=False, ylog=False,
                step=False, std=False,
                **kwargs):
    """
    Plot a series of data with ordered x axis (e.g.: epoch series).
    
    Required arguments:
        ax:   the subplot ax where to plot data
        data: the data to plot
        
    Optional arguments:
        title:    the title of the plot
        xlabel:   the label of the x axis
        ylabel:   the label of the y axis
        legend:   the label for the legend in the plot
        xlog:     whether to use the log scale on the x axis
        ylog:     whether to use the log scale on the y axis
        step:     whether to use a step function for the plot
        std:      highlight the strip of the standard deviation
        **kwargs: additional arguments to pass to plt.step or plot.plot
    """

    ax.grid(alpha=0.2)                   # create the grid
    ax.set_title(title)                  # set the title
    ax.set_xlabel(xlabel)                # set labels for the x axis
    ax.set_ylabel(ylabel)                # set labels for the y axis

    if xlog:                             # use log scale in the x axis if needed
        ax.set_xscale('log')
    if ylog:                             # use log scale in the y axis if needed
        ax.set_yscale('log')

    series = np.arange(len(data))        # create trivial x axis data
    if step:                             # create the plot
        ax.step(series, data, label=legend, **kwargs)
    else:
        ax.plot(series, data, label=legend, **kwargs)

    if std:                              # show coloured strip with std
        ax.fill_between(series,
                        data + np.std(data),
                        data - np.std(data),
                        alpha=0.2)

    if legend is not None:               # show the legend
        ax.legend(loc='best')

    return ax

Before going further we also set the **memory growth** of the GPU RAM in order to avoid memory issues:

In [9]:
# get the list of installed GPUs
gpus = tf.config.experimental.list_physical_devices('GPU')

if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True) # set memory growth
            
        # get the list of logical devices (GPUs)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        logprint('GPU setup: {:d} physical GPUs, {:d} logical GPUs.'.format(len(gpus), len(logical_gpus)), logger=logger)
    except RuntimeError as e:
        logprint(e, stream='error', logger=logger)
else:
    logprint('No GPUs in the setup!', stream='error', logger=logger)

2020-04-09 09:30:33,266 --> ERROR: No GPUs in the setup!


## Neural Networks

We then proceed with the analysis by first loading the prepared datasets and then building the appropriate architectures for the prediction of the Hodge numbers. We start from the analysis of the configuration matrix through a **CNN** architecture and then compare the results with the analysis of the engineered dataset.

We consider only the configuration matrix and build a CNN to predict the labels. First of all we load the dataset and then divide labels and features, we then proceed with the architecture of the CNN.

In [20]:
if path.isfile(path.join(ROOT_DIR, DB_NAME + '_matrix.h5')):
    matrix = load_dataset(path.join(ROOT_DIR, DB_NAME + '_matrix.h5'), logger=logger)
    h11    = ExtractTensor(flatten=False).fit_transform(matrix['h11'])
    h21    = ExtractTensor(flatten=False).fit_transform(matrix['h21'])
    matrix = ExtractTensor(flatten=False).fit_transform(matrix['matrix'])

2020-04-09 09:40:09,455 --> INFO: Reading database...
2020-04-09 09:40:09,512 --> INFO: Database loaded!


## Setup and Preparation of the Input

Before splitting the dataset into training and test sets, we need to resize the feature inputs in order to feed them to a convolutional layer.

In [ ]:
def reshape_matrix(matrix):
    """
    Reshape the input given as a matrix (rows x cols) into (rows x cols x depth) where depth = 1
    
    Required arguments:
        matrix: the matrix to reshape
        
    Returns:
        the reshaped matrix
    """
    
    rows = np.shape(matrix)[0]
    cols = np.shape(matrix)[1]
    
    reshaped_matrix = []
    
    for m in matrix:
        reshaped_matrix.append(np.reshape(matrix, (rows, cols, 1)))
        
    return reshaped_matrix    

We then apply it to the entire dataset of matrices:

In [ ]:
logprint('Reshaping the matrix...', logger=logger)
matrix = reshape_matrix(matrix)
logprint('Matrix reshaped!', logger=logger)

As for the ML analysis, we keep 10% of the whole dataset as **test** set, while we use 90% of it as **training** set. Specifically we then retain 1/9 of the training set as holdout **validation** such that the entire dataset is thus divided into 10% for test predictions, 10% for validation and 80% for effective training. 

In [ ]:
from sklearn.model_selection import train_test_split

logprint('Splitting into training and test sets...', logger=logger)
matrix_train, matrix_test, h11_train, h11_test, h21_train, h21_test = train_test_split(matrix, h11, h21,                   test_size=0.1,     shuffle=False)
matrix_train, matrix_val,  h11_train, h11_val,  h21_train, h21_val  = train_test_split(matrix_train, h11_train, h21_train, test_size=1.0/9.0, shuffle=False)
logprint('Divided into training and test sets!', logger=logger)

## Model Building

We the build the architecture of the CNN. We use a modular approach in order to be able to change its parameters in a quick way in order to perform the hyperparameter optimization efficiently. We build the network using blocks of **convolutional layers** (`conv_n_blocks`) with several hidden layers inside (`conv_n_hidden_layers`) with a decreasing number of filters in them (`conv_n_initial_filters`) and the **pooling** operation at the end (`conv_pooling`, can be _avg_ for _AvgPool2D_ or _max_ for _MaxPool2D_, with associated `conv_pool_size` for the size of the pooling). The kernel size is decreasing as well and starts at a given size (`conv_kernel_size`) with padding. We then attach the CNN to a **fully connected** (FC) network with blocks of **dense layers** (`dense_n_hidden_layers`) with a decreasing number of neurons in them (`dense_n_initial_neurons`). We also use **batch normalization** (`batch_normalization`, can be true or false) to improve the result and we add **dropout layers** to avoid overfitting (`dropout`, can be _all_ for dropout after every block or _final_ for dropout only before the FC network, and `dropout_rate` for its rate). We also give the possibility to change the activation function for the convolutional and dense layers (`conv_activation`, can be _leaky_ for _LeakyReLU_ or _relu_ for _ReLU_, and `dense_activation`, can be _leaky_ for _LeakyReLU_ or _relu_ for _ReLU_. For the _LeakyReLU_ activation, we also consider `leaky_alpha` for its slope). Since we want to predict $h_{11} \ge 0$ and $h_{21} \ge 0$, we also consider whether to use the _ReLU_ activation function in the output layer to force a positive output (`final_relu`, can be true or false). The model creation also involves the learning rate which should be chosen carefully (`learning_rate`).

In [ ]:
from tensorflow.keras            import losses, metrics
from tensorflow.keras.models     import Model
from tensorflow.keras.layers     import Lambda, Conv2D, Dense, Flatten, MaxPool2D, AvgPool2D, Activation, LeakyReLU, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam

def cnn_model(input_shape,
              conv_n_blocks=1,
              conv_n_hidden_layers=1,
              conv_n_initial_filters=16,
              conv_kernel_size=3,
              conv_activation='relu',
              conv_pooling='max',
              conv_pool_size=2,
              dense_n_hidden_layers=1,
              dense_n_initial_neurons=10,
              dense_activation='relu',
              leaky_alpha=0.5,
              final_relu=False,
              batch_normalization=True,
              dropout='final',
              dropout_rate=0.2,
              learning_rate=0.001
             ):
    """
    Build a CNN model with blocks of convolutional layers attached to a FC network.
    
    Required arguments:
        input_shape:             the Numpy shape of the input tensor
    
    Optional arguments:
        conv_n_blocks:           the number of convolutional blocks
        conv_n_hidden_layers:    the number of convolutional layers in each block
        conv_n_initial_filter:   the number of filters in the first convolutional block
        conv_kernel_size:        the size of the kernel in the first convolutional block
        conv_activation:         the activation function of the convolutional layers
        conv_pooling:            the pooling operation at the end of each block
        conv_pool_size:          the size of the pooling window
        dense_n_hidden_layers:   the number of hidden layers in the FC network
        dense_n_initial_neurons: the number of neurons in the first FC layer
        dense_activation:        the activation function of the FC network
        leaky_alpha:             the slope of the LeakyReLU activation
        final_relu:              whether to use ReLU in the output layer
        batch_normalization:     whether to use batch normalization
        dropout:                 what kind of dropout strategy to use
        dropout_rate:            the rate of the dropout
        learning_rate:           the learning rate for the gradient descent
        
    Accepted inputs:
        conv_n_blocks:           (int)   strictly positive integer (cannot be 0)
        conv_n_hidden_layers:    (int)   strictly positive integer (cannot be 0)
        conv_n_initial_filters:  (int)   strictly positive integer (cannot be 0)
        conv_kernel_size:        (int)   strictly positive integer (cannot be 0)
        conv_activation:         (str)   'relu' for ReLU or 'leaky' for LeakyReLU (see 'leaky_alpha')
        conv_pooling:            (str)   'max' for MaxPool, 'avg' for AvgPool or None
        conv_pool_size:          (int)   strictly positive integer (must be greater than 1)
        dense_n_hidden_layers:   (int)   strictly positive integer (cannot be 0)
        dense_n_initial_neurons: (int)   strictly positive integer (cannot be 0)
        dense_activation:        (str)   'relu' for ReLU or 'leaky' for LeakyReLU (see 'leaky_alpha')
        leaky_alpha:             (float) strictly positive floating point number
        final_relu:              (bool)  True or False
        batch_normalization:     (bool)  True or False
        dropout:                 (str)   'final' for dropout only before the FC network, 'all' for dropout after every block
        dropout_rate:            (float) strictly positive floating point number (< 1.0)
        learning_rate:           (float) strictly positive floating point number
    """
    
    # compute the space of convolutional filters and the kernel sizes, based on the no. of blocks and the no. of initial filters
    filters = np.linspace(1, conv_n_initial_filters, conv_n_blocks, dtype=np.int8)
    kernels = np.linspace(1, conv_kernel_size,       conv_n_blocks, dtype=np.int8)
    
    # compute the space of fully connected neurons used in each dense layer, based on the no. of blocks and the no. of initial filters
    neurons = np.linspace(2, dense_n_initial_neurons, dense_n_hidden_layers, dtype=np.int8)
    
    # build the convolutional model
    i = Input(shape=input_shape) #---------------------------------------------------------- INPUT layer
    x = Lambda(lambda x: x)(i)
    
    for n_block in range(conv_n_blocks): #-------------------------------------------------- START convolutional block
        
        x = BatchNormalization()(x) if batch_normalization #------------------------------------ ADD batch normalization
        
        for n_layer in range(conv_n_hidden_layers): #------------------------------------------- START hidden convolutional layers
            
            x = Conv2D(filters=filters[conv_n_blocks - 1 - n_block],
                       kernel_size=kernels[conv_n_blocks - 1 - n_block],
                       padding='same'
                      ) #--------------------------------------------------------------------------- ADD convolutional layer
            
            x = Activation('relu') if   conv_activation == 'relu' \
                                   else LeakyReLU(alpha=leaky_alpha) #------------------------------ ADD activation function
            
            #----------------------------------------------------------------------------------- END hidden convolutional layers
            
        if conv_pooling is not None and \
        (x.output_shape[0] > conv_pool_size and x.output_shape[1] > conv_pool_size):
            
            x = MaxPool2D(pool_size=conv_pool_size, \
                          padding='same') if conv_pooling == 'max' \
                                          else AvgPool2D(pool_size=conv_pool_size, \
                                                         padding='same') #---------------------- ADD pooling (if allowed and requested)
            
        if dropout == 'all':
            x = Dropout(rate=dropout_rate)(x) #------------------------------------------------- ADD dropout (if requested)
            
        #----------------------------------------------------------------------------------- END convolutional block
        
    x = Flatten()(x) #---------------------------------------------------------------------- START fully connected network

    x = Dropout(rate=dropout_rate)(x) #----------------------------------------------------- ADD dropout
    
    for n_layer in range(dense_n_hidden_layers): #------------------------------------------ START hidden dense layers
        
        x = BatchNormalization()(x) if batch_normalization #------------------------------------ ADD batch normalization
        
        x = Dense(units=neurons[dense_n_hidden_layers - 1 - n_layer])(x) #---------------------- ADD dense layer
        
        x = Activation('relu') if dense_activation == 'relu' \
                               else LeakyReLU(alpha=leaky_alpha) #------------------------------ ADD activation layer
        
        if dropout == 'all':
            x = Dropout(rate=dropout_rate)(x) #------------------------------------------------- ADD dropout (if requested)
            
        #----------------------------------------------------------------------------------- END hidden dense layers
        
    f = Dense(1)(x) #----------------------------------------------------------------------- OUTPUT layer
    
    f = Activation('relu') if final_relu #-------------------------------------------------- OUTPUT layer (w/ activation)
    
    # create the model and compile it
    model = Model(inputs=i, outputs=f)
    model.compile(optimizer=Adam(learning_rate=learning_rate, loss=losses.MeanSquaredError, metrics=[metrics.MeanSquaredError]))
    
    return model

We then consider the function to maximize, namely the accuracy of the model:

In [ ]:
import json

from skopt                       import gp_minimize
from skopt.plots                 import plot_convergence
from skopt.space                 import Categorical, Integer, Real
from skopt.utils                 import use_named_args
from tensorflow.keras.callbacks  import EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils      import plot_model

# decide the input shape
input_shape = np.shape(matrix[0])

# decide the number of samplings to perform on the search space
n_iter = 10

# create search space
search_params = [Integer(1,  5,                      prior='uniform',     name='conv_n_blocks'),
                 Integer(1,  5,                      prior='uniform',     name='conv_n_hidden_layers'),
                 Integer(16, 512,                    prior='uniform',     name='conv_n_initial_filters'),
                 Integer(2,  6,                      prior='uniform',     name='conv_kernel_size'),
                 Integer(2,  4,                      prior='uniform',     name='conv_pool_size'),
                 Integer(0,  10,                     prior='uniform',     name='dense_n_hidden_layers'),
                 Integer(1e0, 1e3,          base=10, prior='log-uniform', name='dense_n_initial_neurons'),
                 Real(0.1, 0.5,                      prior='uniform',     name='dropout_rate'),
                 Real(1.0e-2, 1.0e1,        base=10, prior='log-uniform', name='leaky_alpha'),
                 Real(1.0e-3, 1.0e-1,       base=10, prior='log-uniform', name='learning_rate'),
                 Integer(False, True,                                     name='final_relu'),
                 Integer(False, True,                                     name='batch_normalization'),
                 Categorical(['relu', 'leaky'],                           name='conv_activation'),
                 Categorical(['relu', 'leaky'],                           name='dense_activation'),
                 Categorical(['max', 'avg', None],                        name='conv_pooling'),
                 Categorical(['all', 'final'],                            name='dropout')
                ]

We then fit the model for the first Hodge number, namely $h_{11}$:

In [ ]:
@use_named_args(search_params)
def objective_h11(**params):
    
    # create model
    logprint('Building a CNN model...', logger=logger)
    model = cnn_model(input_shape=input_shape, **params)
    
    # print summary
    model.summary()
    
    # fit the model (for the moment no checkpoint save needed: once we have the best parameters, then we train and save the best)
    logprint('Fitting a CNN model...', logger=logger)
    model.fit(x=matrix_train,
              y=h11_train,
              batch_size=32,
              epochs=500,
              verbose=1,
              callbacks=EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True),
              validation_data=(matrix_val, h11_val)
             )
    
    # evaluate the model on the validation set and return accuracy (actually 1 - acc to be minimized)
    logprint('Evaluating model...', logger=logger)
    predictions = model.predict(matrix_val)
    return 1.0 - accuracy_score(predictions, h11_val, rounding=np.rint)

# compute the minimization
model_h11_res = gp_minimize(objective_h11, search_params, n_calls=n_iter, random_state=RAND)

# return the best parameters and print them
best_params = {search_params[n].name: model_h11_res.x[n] for n in range(len(model_h11_res.x))}
model_h11   = cnn_model(input_shape=input_shape, **best_params)
model_h11_json = model_h11.to_json()
with open(path.join(MOD_PATH, 'cnn_matrix_h11_arch.json'), 'w') as f:
    json.dump(model_h11_json, f)

# fit the best model on the training set and compute the accuracy of the predictions
logprint('Best parameters: {}'.format(best_params), logger=logger)
model_h11.fit(x=matrix_train,
              y=h11_train,
              batch_size=32,
              epochs=500,
              verbose=1,
              callbacks=[EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True),
                         ModelCheckpoint(filepath=path.join(MOD_PATH, 'cnn_matrix_h11_weights.h5'), monitor='val_loss', save_best_only=True, save_weights_only=True)
                        ],
              validation_data=(matrix_val, h11_val)
             )

preds_score_h11 = prediction_score(estimator=model_h11, X=matrix_test, y=h11_test, use_best_estimator=False, rounding=np.rint, logger=logger)

And then repeat the same procedure for the second Hodge number, $h_{21}$:

In [ ]:
@use_named_args(search_params)
def objective_h21(**params):
    
    # create model
    logprint('Building a CNN model...', logger=logger)
    model = cnn_model(input_shape=input_shape, **params)
    
    # print summary
    model.summary()
    
    # fit the model (for the moment no checkpoint save needed: once we have the best parameters, then we train and save the best)
    logprint('Fitting a CNN model...', logger=logger)
    model.fit(x=matrix_train,
              y=h21_train,
              batch_size=32,
              epochs=500,
              verbose=1,
              callbacks=EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True),
              validation_data=(matrix_val, h21_val)
             )
    
    # evaluate the model on the validation set and return accuracy (actually 1 - acc to be minimized)
    logprint('Evaluating model...', logger=logger)
    predictions = model.predict(matrix_val)
    return 1.0 - accuracy_score(predictions, h21_val, rounding=np.rint)

# compute the minimization
model_h21_res = gp_minimize(objective_h21, search_params, n_calls=n_iter, random_state=RAND)

# return the best parameters and print them
best_params = {search_params[n].name: model_h21_res.x[n] for n in range(len(model_h21_res.x))}
model_h21   = cnn_model(input_shape=input_shape, **best_params)
model_h21_json = model_h21.to_json()
with open(path.join(MOD_PATH, 'cnn_matrix_h21_arch.json'), 'w') as f:
    json.dump(model_h21_json, f)

# fit the best model on the training set and compute the accuracy of the predictions
logprint('Best parameters: {}'.format(best_params), logger=logger)
model_h21.fit(x=matrix_train,
              y=h21_train,
              batch_size=32,
              epochs=500,
              verbose=1,
              callbacks=[EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True),
                         ModelCheckpoint(filepath=path.join(MOD_PATH, 'cnn_matrix_h21_weights.h5'), monitor='val_loss', save_best_only=True, save_weights_only=True)
                        ],
              validation_data=(matrix_val, h21_val)
             )

preds_score_h21 = prediction_score(estimator=model_h21, X=matrix_test, y=h21_test, use_best_estimator=False, rounding=np.rint, logger=logger)